# Importaciones

In [ ]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from google.colab import files

In [ ]:
# Descargamos el Dataset desde el enlace
url = "https://drive.google.com/uc?export=download&id=1Ds68fJvZK9MlVxeWQ2S7M4_v-PibzOqN"
# Leemos el Dataset
df = pd.read_csv(url)

# Preprocesamiento del DataFrame

In [ ]:
# Exploración inicial
print("Dimensiones del dataset:", df.shape)
print("Primeras filas:")
df.head()

In [ ]:
# Elimino la columna Student_ID porque no nos aporta nada
df = df.drop(columns=["Student_ID"])
df.head()

In [ ]:
# Reemplazo los valores de Stress_Level por números
stress_mapping = {"Low": 0, "Moderate": 1, "High": 2}
df["Stress_Level"] = df["Stress_Level"].map(stress_mapping)

# Verificar resultado
df[["Stress_Level"]].head()

In [ ]:
# Reviso si hay valores nulos
df.isnull().sum()

In [ ]:
# Veo en detalle el DataFrame
print("Resumen estadístico:")
df.describe()

In [ ]:
# Visualizamos los datos de cada variable para ver los outliners

cols = [
    "Study_Hours_Per_Day",
    "Extracurricular_Hours_Per_Day",
    "Sleep_Hours_Per_Day",
    "Social_Hours_Per_Day",
    "Physical_Activity_Hours_Per_Day",
    "GPA",
    "Stress_Level"
]

# Crear figura con subplots
plt.figure(figsize=(15, 10))

for i, col in enumerate(cols, 1):
    plt.subplot(3, 3, i)
    sns.boxplot(y=df[col])
    plt.title(col)

plt.tight_layout()
plt.show()

# Análisis introductorio

In [ ]:
# Imprimo la media y los valores máximos y mínimos del GPA
mean_gpa = float(df["GPA"].mean())
gpa_min = float(df["GPA"].min())
gpa_max = float(df["GPA"].max())

print("Media de GPA:",mean_gpa)
print("GPA más bajo:", gpa_min)
print("GPA más alto:", gpa_max)

In [ ]:
# Creamos un gráfico matriz de relación donde se vean las correlaciones entre las variables

# Calcular la matriz de correlación entre todas las variables
corr_all = df[["Study_Hours_Per_Day", "Stress_Level", "Sleep_Hours_Per_Day", "Extracurricular_Hours_Per_Day", "Social_Hours_Per_Day", "Physical_Activity_Hours_Per_Day", "GPA"]].corr()

# Graficar heatmap
plt.figure(figsize=(7, 6))
sns.heatmap(corr_all, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlaciones entre variables')
plt.show()

In [ ]:
# Creamos un gráfico donde se muestre el promedio de horas de sueño según nivel de estrés

# Calcular promedio de horas de sueño por nivel de estrés
stress_sleep = df.groupby("Stress_Level")["Sleep_Hours_Per_Day"].mean().reset_index()

# Valores mínimo y máximo de todo el dataset
min_sleep = df["Sleep_Hours_Per_Day"].min()
max_sleep = df["Sleep_Hours_Per_Day"].max()

# Gráfico de barras
plt.figure(figsize=(6,5))
ax = sns.barplot(
    x="Stress_Level",
    y="Sleep_Hours_Per_Day",
    data=stress_sleep,
    hue="Stress_Level",
    palette="Set2",
    legend=False
)

plt.title("Promedio de horas de sueño según nivel de estrés")
plt.xlabel("Nivel de Estrés")
plt.ylabel("Horas de Sueño (promedio)")

# Escala ajustada al rango real de los estudiantes
plt.ylim(min_sleep, max_sleep)

# Añadir valores encima de cada barra
for p in ax.patches:
    height = p.get_height()
    ax.text(
        p.get_x() + p.get_width()/2.,
        height + 0.05,
        f"{height:.2f}",
        ha="center"
    )

plt.show()

In [ ]:
# Creamos un gráfico donde se vea la relación entre horas de sueño y horas de estudio según nivel de estrés

plt.figure(figsize=(8,6))
sns.scatterplot(
    data=df,
    x="Study_Hours_Per_Day",
    y="Sleep_Hours_Per_Day",
    hue="Stress_Level",
    palette="Set2",
    s=100, alpha=0.7
)
plt.title("Relación entre horas de sueño y horas de estudio según nivel de estrés")
plt.show()

In [ ]:
# Creamos un gráfico donde se vea la relación entre actividad física, estudio y sueño según el GPA

# Calcular percentiles para dividir GPA en tres grupos
q25 = df['GPA'].quantile(0.25)
q75 = df['GPA'].quantile(0.75)

# Función para asignar grupo según GPA
def gpa_group(gpa):
    if gpa <= q25:
        return 'GPA Bajo'
    elif gpa >= q75:
        return 'GPA Alto'
    else:
        return 'GPA Medio'

# Crear columna temporal con el grupo de GPA
df['GPA_Group'] = df['GPA'].apply(gpa_group)

# Gráfica 3D interactiva
fig = px.scatter_3d(df,
                    x='Physical_Activity_Hours_Per_Day',
                    y='Study_Hours_Per_Day',
                    z='Sleep_Hours_Per_Day',
                    color='GPA_Group',
                    width=1200, height=600,
                    color_discrete_map={
                        'GPA Bajo': 'red',
                        'GPA Medio': 'orange',
                        'GPA Alto': 'green'
                    })

fig.update_traces(marker_size=4)
fig.update_layout(title="Relación entre actividad física, estudio y sueño según el GPA",
                  scene=dict(
                      xaxis_title='Horas de actividad física por día',
                      yaxis_title='Horas de estudio por día',
                      zaxis_title='Horas de sueño por día'
                  ))
fig.show()

# Eliminar la columna temporal para no modificar permanentemente el DataFrame
df.drop(columns=['GPA_Group'], inplace=True)

# Clustering

In [ ]:
# Seleccionar columnas para el clustering
X = df[["Study_Hours_Per_Day", "Extracurricular_Hours_Per_Day",
        "Sleep_Hours_Per_Day", "Social_Hours_Per_Day",
        "Physical_Activity_Hours_Per_Day", "Stress_Level"]]

In [ ]:
# Escalar las variables
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Determinar el número óptimo de clusters

scores = []
for k in range(2, 8):  # probamos de 2 a 7 clusters
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    scores.append(score)

optimal_k = scores.index(max(scores)) + 2
print("Número óptimo de clusters:", optimal_k)

In [ ]:
# Crear el modelo usando el número óptimo de clusters

kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df['Cluster'] = kmeans_final.fit_predict(X_scaled)

# Revisar cuántos estudiantes hay en cada cluster
print(df['Cluster'].value_counts())

In [ ]:
df['Cluster'].head(5)

# Análisis tras la asignación de los Clusters

In [ ]:
# Creamos un gráfico donde para el Cluster 1 se vea la relación entre horas de estudio y horas de actividad física según el nivel de estrés

# Filtrar los estudiantes del Cluster 1
cluster1_df = df[df['Cluster'] == 1]

# Gráfico de dispersión con color según nivel de estrés
plt.figure(figsize=(8,6))
sns.scatterplot(
    data=cluster1_df,
    x='Study_Hours_Per_Day',
    y='Physical_Activity_Hours_Per_Day',
    hue='Stress_Level',
    palette='Set2',
    s=80
)

plt.title('Cluster 1: Relación entre horas de estudio y horas de actividad física según el nivel de estrés')
plt.xlabel('Horas de estudio por día')
plt.ylabel('Horas de actividad física por día')
plt.legend(title='Nivel de estrés')
plt.show()

In [ ]:
# Creamos un gráfico donde para el Cluster 2 se vea la relación entre horas de sueño y GPA mostrando el porcentaje de estudiantes que se encuentra en cada rango horario

# Filtrar estudiantes del Cluster 2
cluster2_df = df[df['Cluster'] == 2]

# Definir rangos de horas de sueño
ranges = [(0, 5.99), (6, 7.49), (7.5, 8.99), (9, cluster2_df['Sleep_Hours_Per_Day'].max()+0.5)]
colors = ['#FFC1C1', '#FFE4B5', '#C1FFC1', '#ADD8E6']

# Calcular porcentajes
percentages = []
for low, high in ranges:
    count = cluster2_df[(cluster2_df['Sleep_Hours_Per_Day'] >= low) &
                        (cluster2_df['Sleep_Hours_Per_Day'] <= high)].shape[0]
    percentages.append(count / cluster2_df.shape[0] * 100)

# Crear gráfico
plt.figure(figsize=(10,6))

# Dibujar bandas
min_x = cluster2_df['Sleep_Hours_Per_Day'].min() - 0.5
max_x = cluster2_df['Sleep_Hours_Per_Day'].max() + 0.5
min_y = cluster2_df['GPA'].min()
max_y = cluster2_df['GPA'].max()

for (low, high), color in zip(ranges, colors):
    plt.axvspan(low, high, color=color, alpha=0.4)

# Graficar puntos
plt.scatter(
    cluster2_df['Sleep_Hours_Per_Day'],
    cluster2_df['GPA'],
    color='blue'
)

# Ajustes de ejes
plt.xlim(min_x, max_x)
plt.ylim(min_y-0.3, max_y+0.2)

# Posición vertical para los porcentajes
y_offset = min_y - 0.15

# Dibujar porcentajes centrados en cada rango
for (low, high), pct in zip(ranges, percentages):
    visible_low = max(low, min_x)
    visible_high = min(high, max_x)
    center_x = (visible_low + visible_high) / 2

    plt.text(center_x, y_offset, f'{pct:.1f}%',
             ha='center', va='top', fontsize=12, color='black')

plt.xlabel('Horas de sueño por día')
plt.ylabel('GPA')
plt.title('Cluster 2: Relación entre horas de sueño y GPA')
plt.show()


In [ ]:
# Creamos un gráfico donde se vea la distribución de Clusters dentro de cada rango de resultados de GPA

# Calcular percentiles de GPA
q25 = df['GPA'].quantile(0.25)
q75 = df['GPA'].quantile(0.75)

def gpa_group(gpa):
    if gpa <= q25:
        return 'GPA Bajo'
    elif gpa >= q75:
        return 'GPA Alto'
    else:
        return 'GPA Medio'

df['GPA_Group'] = df['GPA'].apply(gpa_group)

# Colores de clusters
cluster_colors = {0:'red', 1:'green', 2:'blue', 3:'orange'}

fig, axes = plt.subplots(1, 3, figsize=(18,6))

for ax, group_name in zip(axes, ['GPA Bajo', 'GPA Medio', 'GPA Alto']):
    subset = df[df['GPA_Group'] == group_name]
    cluster_counts = subset['Cluster'].value_counts()
    percentages = cluster_counts / cluster_counts.sum() * 100

    # Ordenar de mayor a menor
    percentages = percentages.sort_values(ascending=False)

    # Gráfico de pastel
    ax.pie(
        percentages,
        labels=None,
        colors=[cluster_colors[c] for c in percentages.index],
        startangle=90
    )
    ax.set_title(group_name)
    ax.axis('equal')

    text_lines = [f"{percentages[c]:.1f}% Cluster {c}" for c in percentages.index]
    text = "\n".join(text_lines)
    ax.text(0, -1.2, text, ha='center', va='top',
        bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))

# Leyenda de colores de clusters
handles = [plt.Line2D([0], [0], marker='s', color='w', label=f'Cluster {c}',
                      markerfacecolor=color, markersize=15)
           for c, color in cluster_colors.items()]
fig.legend(handles=handles, loc='upper right')

plt.suptitle('Distribución de Clusters dentro de resultados de GPA')
plt.show()

# Eliminar columna temporal
df.drop(columns=['GPA_Group'], inplace=True)

In [ ]:
# Creamos un gráfico donde se vea la distribución de niveles de estrés por Cluster

# Calcular conteos por nivel de estrés dentro de cada cluster (para todos los clusters)
counts = (
    df
    .groupby(['Cluster', 'Stress_Level'])
    .size()
    .unstack(fill_value=0)
)

# Asegurarse de que siempre haya columnas para 0,1,2
counts = counts.reindex(columns=[0, 1, 2], fill_value=0)

# Convertir a porcentajes por cluster
percentages = counts.div(counts.sum(axis=1), axis=0) * 100

heatmap_data = percentages.T  # filas estrés (0,1,2), columnas clusters (0,1,2,3)

# Invertir el eje y para que el nivel 2 aparezca arriba
heatmap_data = heatmap_data.iloc[::-1]

plt.figure(figsize=(8,6))
sns.heatmap(
    heatmap_data,
    annot=True, fmt=".1f", cmap="YlOrRd",
    cbar_kws={'label': 'Porcentaje (%)'}
)

plt.title("Distribución de niveles de estrés por Cluster")
plt.xlabel("Cluster")
plt.ylabel("Nivel de Estrés")
plt.yticks(rotation=0)
plt.show()

In [ ]:
# Creamos un gráfico donde se muestren los 100 estudiantes con mayor GPA diferenciados por cluster y su relación entre GPA y horas de estudio

# Colores de los clusters
cluster_colors = {0:'red', 1:'green', 2:'blue', 3:'orange'}

# Seleccionar los 100 estudiantes con mayor GPA
top_students = df.nlargest(100, 'GPA')

plt.figure(figsize=(8,6))
sns.scatterplot(
    data=top_students,
    x="Study_Hours_Per_Day",
    y="GPA",
    hue="Cluster",
    palette=cluster_colors,
    s=100,
    alpha=0.7
)
plt.title("Top 100 estudiantes con mayor GPA: Relación entre GPA y horas de Estudio")
plt.xlabel("Horas de Estudio por Día")
plt.ylabel("GPA")
plt.legend(title="Cluster")
plt.show()


In [ ]:
# Creamos varios gráficos donde se pueda ver la distribución de estudiantes por cada cluster en diferentes habitos y registros

# Lista de atributos
attributes = [
    'GPA',
    'Sleep_Hours_Per_Day',
    'Study_Hours_Per_Day',
    'Social_Hours_Per_Day',
    'Physical_Activity_Hours_Per_Day',
    'Extracurricular_Hours_Per_Day'
]

# Colores de los clusters
cluster_colors = {0:'red', 1:'green', 2:'blue', 3:'orange'}

# Crear subplots
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(18,10))
axes = axes.flatten()

for ax, attr in zip(axes, attributes):
    sns.stripplot(
        data=df,
        x='Cluster',
        y=attr,
        hue='Cluster',         # asignar el cluster como hue
        palette=cluster_colors,
        dodge=False,
        size=6,
        alpha=0.7,
        ax=ax,
        legend=False
    )
    ax.set_title(attr)
    ax.set_xlabel("Cluster")
    ax.set_ylabel(attr)

plt.tight_layout()
plt.show()

# Pasar a CSV

In [ ]:
# Creamos un nuevo Dataframe aparte sin la variable cluster
X = df.drop(columns=["Cluster"])
X = StandardScaler().fit_transform(X) # Escalamos las variables numéricas de este Dataframe aparte

# Reducimos dimensionalidad con PCA
pca = PCA(n_components=2)
pca_coords = pca.fit_transform(X)

# Creamos los atributos PCA1 y PCA2 en el DataFrame original
df["PCA1"] = pca_coords[:, 0]
df["PCA2"] = pca_coords[:, 1]

# Reducimos dimensionalidad con t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30, max_iter=2000)
tsne_coords = tsne.fit_transform(X)

# Creamos los atributos TSNE1 y TSNE2 en el DataFrame original
df["tSNE1"] = tsne_coords[:, 0]
df["tSNE2"] = tsne_coords[:, 1]

In [ ]:
# Exportamos el DataFrame a CSV indicando que las separaciones serán ";" y los decimales serán ","
df.to_csv("clusters_students_visualization.csv", sep=";", decimal=",", index=False)

In [ ]:
# Descargamos el CSV
files.download("clusters_students_visualization.csv")